In [12]:
import sys
import requests
import json
import pandas as pd
from datetime import datetime, timedelta

from pprint import PrettyPrinter
pp = PrettyPrinter(indent=4)



In [13]:
def adjust_sensors_delay_time(implementation):
    """
    This function returns the delay time for each sensor in order to adjust the data
    """
    #esp32
    seconds_delay = 0
    milliseconds_delay = 0
    if implementation == "ESP32S3": 
        seconds_delay = 2
        milliseconds_delay = 920

    return timedelta(seconds=seconds_delay, milliseconds=milliseconds_delay)

In [14]:
def jobs_unix_timestamp(implementation, day, month):
    epoch = datetime.utcfromtimestamp(0)

    #startDT = datetime(2023, month, day, 9, 0, 0, 0) - timedelta(hours=1) + adjust_sensors_delay_time(implementation)
    startDT = datetime(2023, month, day, 9, 0, 0, 0) - timedelta(hours=1) + adjust_sensors_delay_time(implementation)
    startTS = int((startDT - epoch).total_seconds() * 1000.0)
    #endDT = datetime(2023, month, day, 18, 0, 0 , 0) - timedelta(hours=1) + adjust_sensors_delay_time(implementation)
    endDT = datetime(2023, month, day, 18, 0, 0 , 0) - timedelta(hours=1) + adjust_sensors_delay_time(implementation)
    endTS = int((endDT - epoch).total_seconds() * 1000.0)
    

    return str(startTS), str(endTS)

In [15]:
RFID_analysis_keys = [
    'gabarit'
]

TASK_analysis_keys = [
    'task', 'empid', 'sec'
]
TASKTOTAL_analysis_keys = [
    'emp_id', 'taskid','totalm', 'av','pr' 
]
BLE_analysis_keys = [
    'id','rssi'
]


BLE_analysis_keys = [
    'id','rssi'
]


In [16]:
def json_csv(data, dataset, implementation):
    data_df = pd.DataFrame()
    first_flag = True
    
    if dataset == "rfid":
        analysis_keys = RFID_analysis_keys
    elif dataset == "task":
        analysis_keys = TASK_analysis_keys
    else:
        analysis_keys = BLE_analysis_keys



    for analysis_key in analysis_keys:
        if first_flag:
            data_df = pd.DataFrame(data[analysis_key])
            data_df = data_df.rename(columns={'value': analysis_key})
            first_flag = False
        else:
            data_df = data_df.join(pd.DataFrame(data[analysis_key]).set_index('ts'), on='ts')
            data_df = data_df.rename(columns={'value': analysis_key})
    # change the timestamp to seconds
    data_df['ts'] = pd.to_datetime(data_df['ts']/1000,unit='s') + timedelta(hours=1) - adjust_sensors_delay_time(implementation)
    data_df['ts']= data_df['ts'].astype('datetime64[s]')
    return data_df.iloc[::-1]

def getToken():
    username = 'isabel.portugal@tecnico.ulisboa.pt' #email used to create your ThingsBoard account (your email associated with tennant)
    password = 'tHn%Mv1pnmq2av='		#password of your ThingsBoard account (or your tennant account)
    url = 'http://iot.istartlab.tecnico.ulisboa.pt/api/auth/login'
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    loginJSON = {'username': username, 'password': password}
    tokenAuthResp = requests.post(url, headers=headers, json=loginJSON).json()
    #print(tokenAuthResp)
    token = tokenAuthResp['token']
    return token

In [17]:
tkn = getToken()

my_headers = {'X-Authorization':  "Bearer " + tkn, "Content-Type" : "application/json"}
#print(my_headers)
#url to test connection
url = "http://iot.istartlab.tecnico.ulisboa.pt/api/plugins/telemetry/DEVICE/27110c40-2a6b-11ee-9b43-67453a5ac630/keys/timeseries"

response = requests.get(url, headers=my_headers)
print(response) , print(response.text)

my_headers = {'X-Authorization':  "Bearer " + tkn, "Content-Type" : "application/json"}
#print(my_headers)

<Response [404]>
Device with requested id wasn't found!


In [20]:
# CHANGE ONLY THIS
month_int = 10
day_int = 10
#dataset = "rfid" #"task"
dataset = "ble"

if month_int < 10:
    month_str = "0" + str(month_int)
else:
    month_str = str(month_int)
day_str = str(day_int)

startTS, endTS = jobs_unix_timestamp("ESP32S3", day_int, month_int)

limit = "1000000000"

date_filename = dataset + "_" + day_str + "_" + month_str

cnt = 1

if dataset == "rfid":
    devices = ["27110c40-2a6b-11ee-9b43-67453a5ac633","f4765800-2cc5-11ee-a22d-b5d6c11ab119","10fc9700-2cc6-11ee-a22d-b5d6c11ab119"]
    #for analysis_key in RFID_analysis_keys:
    keys = "ts,gabarit"

elif dataset == "task":
    devices = ["502ba360-d205-11ed-a394-2dc30865e203"]
    #for analysis_key in TASK_analysis_keys:
    keys = "task,empid,sec" 

else:
    devices = ["8516caf0-7a71-11ed-a394-2dc30865e203","fe7d6ff0-b52e-11ed-a394-2dc30865e203","4b593330-2498-11ee-a394-2dc30865e203" ]
    #1: fim da linha, 2: pacotes, 3: elevator
    #for analysis_key in BLE_analysis_keys:
    keys = "id,rssi"   

print(keys)

for dev in devices:
    url = "http://iot.istartlab.tecnico.ulisboa.pt/api/plugins/telemetry/DEVICE/" + dev + "/values/timeseries?keys=" + keys + "&startTs=" + startTS + "&endTs=" + endTS + "&limit=" + limit
    response = requests.get(url, headers=my_headers)
    print("response received", response.text)
    data = json.loads(response.text)
    data_df = json_csv(data, dataset, "ESP32S3")
    if dataset == "rfid" or dataset == "ble":
        data_df.to_csv(date_filename + "_" + str(cnt) + ".csv")
    else:
        data_df.to_csv(date_filename + "_" + ".csv")
                   #
    cnt += 1

print("acc done")


id,rssi
response received {"id":[{"ts":1696957202384,"value":"2"},{"ts":1696957190403,"value":"2"},{"ts":1696957190390,"value":"1"},{"ts":1696957142387,"value":"2"},{"ts":1696957142365,"value":"1"},{"ts":1696957136338,"value":"1"},{"ts":1696957124342,"value":"2"},{"ts":1696957118197,"value":"2"},{"ts":1696957094282,"value":"2"},{"ts":1696957094259,"value":"1"},{"ts":1696957088194,"value":"1"},{"ts":1696957082155,"value":"1"},{"ts":1696957064161,"value":"1"},{"ts":1696957058088,"value":"1"},{"ts":1696957028026,"value":"2"},{"ts":1696957022041,"value":"1"},{"ts":1696957016017,"value":"1"},{"ts":1696957010074,"value":"2"},{"ts":1696957004025,"value":"1"},{"ts":1696956992092,"value":"1"},{"ts":1696956992039,"value":"2"},{"ts":1696956985998,"value":"2"},{"ts":1696956961980,"value":"1"},{"ts":1696956949974,"value":"2"},{"ts":1696956937974,"value":"1"},{"ts":1696956931827,"value":"1"},{"ts":1696956913819,"value":"2"},{"ts":1696956895798,"value":"1"},{"ts":1696956883799,"value":"1"},{"ts":1696

In [19]:
print(response.text)

{"id":[{"ts":1696870795487,"value":"2"},{"ts":1696870789370,"value":"1"},{"ts":1696870771354,"value":"1"},{"ts":1696870765367,"value":"1"},{"ts":1696870759346,"value":"1"},{"ts":1696870753386,"value":"1"},{"ts":1696870747344,"value":"2"},{"ts":1696870723314,"value":"2"},{"ts":1696870711308,"value":"2"},{"ts":1696870705304,"value":"2"},{"ts":1696870699318,"value":"1"},{"ts":1696870693293,"value":"2"},{"ts":1696870687285,"value":"1"},{"ts":1696870681296,"value":"1"},{"ts":1696870669317,"value":"1"},{"ts":1696870657279,"value":"1"},{"ts":1696870657261,"value":"2"},{"ts":1696870651297,"value":"1"},{"ts":1696870639251,"value":"1"},{"ts":1696870627239,"value":"1"},{"ts":1696870609234,"value":"1"},{"ts":1696870597223,"value":"1"},{"ts":1696870579208,"value":"1"},{"ts":1696870573199,"value":"2"},{"ts":1696870567221,"value":"2"},{"ts":1696870549197,"value":"2"},{"ts":1696870543178,"value":"2"},{"ts":1696870537216,"value":"1"},{"ts":1696870525174,"value":"1"},{"ts":1696870525161,"value":"2"},{"t